<a href="https://colab.research.google.com/github/Dotking1/Simple-things/blob/main/Dawg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Inputting data**

In [ ]:
import tensorflow as tf


In [ ]:
!pip install tensorflow matplotlib opencv-python pillow

In [ ]:
scam=["confirmshaming","friendspam"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_dir="/content/drive/MyDrive/train"

In [ ]:
args={"labels":"inferred","label_mode":"categorical","batch_size":6,"image_size":(256,256),"seed":1,"validation_split":.3,"class_names":scam}


In [ ]:
train=tf.keras.utils.image_dataset_from_directory(image_dir,
                                            subset="training",
                                            **args)

In [ ]:
test=tf.keras.utils.image_dataset_from_directory(image_dir,
                                            subset="validation",
                                            **args)

In [ ]:
train


  ## ***Loading data into tensor flow***

In [ ]:
first=train.take(1)
first

In [ ]:
images,labels=list(first)[0]
first_image=images[0]

In [ ]:
first_image


In [ ]:
first_image[:3,:3,0]

In [ ]:
from PIL import Image
Image.fromarray(images[0].numpy().astype('uint8'))


In [ ]:
labels[0]


In [ ]:
train=train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test=test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

   ## **Training an initial layer**##

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
model=Sequential([
      layers.Rescaling(1./255),
      layers.Conv2D(16,3,padding="same",activation="relu",input_shape=(256,256,3)),
      layers.Flatten(),
      layers.Dense(128,activation="relu"),
      layers.Dense(len(scam))



])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
history=model.fit(train,
                  validation_data=test,
                  epochs=8,
                  verbose=1)


In [ ]:
model.summary()


In [ ]:
import pandas as pd
history_df=pd.DataFrame.from_dict(history.history)
history_df[["accuracy","val_accuracy"]].plot()



In [ ]:
def train_model(network,epochs=6):
    model=Sequential(network)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
    history=model.fit(train,
                  validation_data=test,
                  epochs=epochs,
    )
    history_df=pd.DataFrame.from_dict(history.history)
    ##history_df[["accuracy","val_accuracy"]].plot()
    return history_df,model


In [ ]:
network=([
      tf.keras.layers.Rescaling(1./255),
      layers.Conv2D(16,4,padding="same",activation="relu",input_shape=(256,256,3)),

      layers.MaxPooling2D(),
      layers.Conv2D(32,4,padding="same",activation="relu",input_shape=(256,256,3)),
      layers.MaxPooling2D(),
      layers.Conv2D(64,4,padding="same",activation="relu",input_shape=(256,256,3)),
      layers.MaxPooling2D(),
      layers.Dropout(.2),
      layers.Flatten(),
      layers.Dense(128,activation="relu"),
      layers.Dense(len(scam))



])
history_df,model=train_model(network)

In [ ]:
history_df[["accuracy","val_accuracy"]].plot()


## **Augmenting Data**

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.RandomFlip("horizontal",seed=1),
    layers.RandomRotation(.2,seed=1),
    layers.RandomZoom(.2,seed=1)






] )

In [ ]:
history_df[["accuracy","val_accuracy"]].plot()

## **Checking for errors**

In [ ]:
preds=model.predict(test)



In [ ]:
preds



In [ ]:
import numpy as np
predicted_class=np.argmax(preds,axis=1)

In [ ]:
predicted_class


In [ ]:
actual_labels=np.concatenate([y for x,y in test],axis=0)

In [ ]:
actual_labels


In [ ]:
actual_class=np.argmax(actual_labels,axis=1)

In [ ]:
actual_class

In [ ]:
import itertools

actual_image=[x.numpy().astype("uint8") for x,y in test]
actual_image=list(itertools.chain.from_iterable(actual_image))
actual_image=[Image.fromarray(a) for a in actual_image]


In [ ]:
#!pip install Pillow
#from PIL import Image
#actual_image=[Image.fromarray(a) for a in actual_image]
#actual_image=list

In [ ]:
pred_df=pd.DataFrame(zip(predicted_class,actual_class,actual_image),columns=["prediction","actual","image"])

In [ ]:
pred_df["prediction"]=pred_df["prediction"].apply(lambda x: scam[x])
pred_df["actual"]=pred_df["actual"].apply(lambda x:scam[x])

In [ ]:
pred_df.head()

In [ ]:
import base64
import io
from PIL import Image
import cv2
import pandas as pd

def image_formatter(img):
    with io.BytesIO() as buffer:
        img.save(buffer,format='png')
        img_str=base64.b64encode(buffer.getvalue()).decode()
        return f'<img src="data:image/jpeg;base64,{img_str}">'

pred_df.head(10).style.format({'image':image_formatter})
#cv2.imwrite(path,img_to_save)